In [35]:
# a. (5 points) includes one markdown cell introducing, in one paragraph, the data and API that you are testing, any hypotheses you have about how the API will perform, and any other background information that would be helpful to a reader.
# b. (15 points) calls the API to generate the desired output (e.g., text transcriptions of speech-to-text APIs, image classification for computer vision APIs, etc.)
# c. (15 points) includes one code cell block analyzing the resulting output (e.g., combine your API output into one dataframe; merge it to your “ground truth” dataframe; compare the API outputs to your “ground truth” for each row; report statistics / aggregate as necessary)
# i. Comparison between ground truth and API outputs should use a reasonable “metric” that you should substantiate. As examples, Word Error Rate (WER) is a commonly used metric for speech-to-text systems; metrics like classification accuracy, precision, or recall might be relevant for facial recognition systems.
# d. (15 points) includes one markdown cell describing, in one paragraph, your observations about the API performance on your data. For example, if you have multiple classes, do you see differences in your metrics? Do you have guesses for why the API performed well or not well on certain data?
# e. Sources cited (does not need to be in ACM format), per our course policy

I recorded the speech myself. I read paragraphs from 10 New York Times article. The recordings are between 53 s and 59 s. The articles were in the following categories: politics (4 articles); weather (2 articles); health (2 articles); and art and culture (2 articles). I picked articles from different categories so that there would be a wider variety of words in the input data. Another thing to note about the speech being input is that I am an international student and do not have a standard American accent. I used the Google Speech-to-Text API. Certain articles that I read contained transliteration of foreign words and names that I read aloud. My hypothesis is that the API will underperform for these articles/recordings (e.g. Recording_2 and Recording_6). Additionally, the API will struggle with understanding accented speech so it is unlikely we will get a high (90% +) word match. I did not use an uniform speed to record all these articles. The speed varied a lot between articles. For example, I read the article quite fast for Recording_9. My hypothesis is that the API will not struggle with faster speech.

In [38]:
# Install google cloud speech

!pip install google.cloud.speech


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [39]:
# Install Levenshtein
pip install python-Levenshtein

Note: you may need to restart the kernel to use updated packages.


In [40]:
# Install Pandas
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [41]:
import pandas as pd

In [42]:
# Create a DataFrame from a Dictionary
recording_1_text = 'Is There Something Radical About Painting Flowers?Many artists who made their names in figurative work are now creating a different sort of portrait. In Early 2021, Jordan Casteel, the painter known for her tender, exquisitely detailed oil portraits of Black men and other people of color in and around her Harlem community, made two unexpected moves. First, she relocated from New York City to a rural part of the Catskills, a far less populous place where she barely knew anyone. The second move, this one artistic but no less momentous, was that Casteel, 35, began began painting pictures of flowers. The first floral painting she made after arriving upstate was born of a moment of disorientation: Casteel was trying to determine how her practice would work, “being so far from people, and so far from a familiar landscape,” she says. While mulling over this predicament, she was spending hours tending her garden'
recording_2_text = 'What It’s Like to Be a Sociopath Sociopaths are modern-day boogeymen, and the word “sociopath” is casually tossed around to describe the worst, most amoral among us. But they are not boogeymen; they are real people and, according to Patric Gagne, widely misunderstood. Gagne wrote “Sociopath,” her buzzy forthcoming memoir, to try to correct some of those misunderstandings and provide a fuller picture of sociopathy, which is now more frequently referred to as antisocial personality disorder. As a child, Gagne found herself compelled toward violent outbursts in an effort to try to compensate for the emotional apathy that was her default. As she got older, those compulsive behaviors turned into criminal ones like trespassing and theft. Eventually, she discovered that there was a name — that dreaded word — that could be used to describe and explain her'
recording_3_text = 'Judge Orders Trump Removed From Illinois Primary Ballot The judge, a Democrat, stayed her ruling until Friday, leaving Donald J. Trump’s team time to appeal the decision. A state judge in Illinois ruled on Wednesday that former President Donald J. Trump had engaged in insurrection and was ineligible to appear on the state’s primary ballot. The decision creates uncertainty for the state’s March election, in which early voting is already underway. It also adds urgency for the U.S. Supreme Court to provide a national answer to the questions that have been raised about Mr. Trump’s eligibility to appear on ballots in more than 30 states.The judge, Tracie R. Porter of the State Circuit Court in Cook County, said the State Board of Elections had erred in rejecting an attempt to remove Mr. Trump and said the board shall remove Donald J. Trump from the ballot for the general primary election on March 19, 2024'
recording_4_text = 'In Chicago, It’s Summer in February An unseasonably warm winter peaked on Tuesday with melting ice rinks and 70-degree weather. Temperatures will plummet from there, but even so, this winter without winter has left some people rattled.February is usually frigid perfection for the ice rink at Millennium Park in downtown Chicago, a favorite winter stop for tourists and local families that stands in the shadow of the reflective sculpture known as the Bean. On Tuesday morning, the rink was melting. Under an intense sun and 70-degree air temperatures, water slowly trickled out of the empty rink, flooding the surrounding concrete. Baby birds splashed happily in the pools of water. The ticket counter was abandoned, apparently closed for the day.'
recording_5_text = 'Welcome to Japan, Where the Bad News Is the Good News The trend lines may be grim, but the Japanese aren’t sweating them. The economy is now in recession after barely growing for decades. The population continues to shrink, with births last year plunging to a nadir. The country’s politics appear frozen as one party holds a virtual lock on power no matter how scandal-tainted and unpopular it becomes.But not to worry. This is Japan, where all bad news is relative.Take a look around. There are few signs of the societal discord you might expect in a place with trend lines like Japan’s, such as accumulating garbage, potholes or picket lines. The country remains remarkably stable and cohesive, with little sense of impending doom. That equanimity reflects a no-need-to-rock-the-boat mind-set: Shouganai — it can’t be helped — is something of a national refrain.'
recording_6_text = 'House passes Stopgap Bill to Head Off Shutdown, With Key Issues Unresolved.The latest temporary spending measure buys Congress a week to finalize details of a deal on half of government spending, and another three weeks to negotiate and pass the other half. Congress passed its latest short-term stopgap spending patch on Thursday to head off a partial government shutdown at the end of the week, giving lawmakers more time to resolve funding disputes that have persisted for months.The measure, approved first by the House and hours later by the Senate, would extend funding for half of the government for one week, through March 8, and the rest for three weeks, until March 22. President Biden is expected to quickly sign it, averting a lapse in federal funding for several agencies that otherwise would begin at 12:01 a.m. on Saturday.'
recording_7_text = 'Why High Blood Pressure Matters to Your Health And how to get it under control.Nearly half of American adults have high blood pressure — but only a quarter of those with hypertension have it under control, according to the Centers for Disease Control and Prevention.High blood pressure “is a smoldering public health crisis,” said Dr. Rishi K. Wadhera, an associate professor of medicine at Harvard Medical School and an author of new research showing that blood pressure screenings have not returned to what they were before the coronavirus pandemic.High blood pressure can raise the risk of heart attack, stroke, pregnancy complications and other health problems, but the symptoms are “often silent,” Dr. Wadhera said. “I worry it flies under the radar for many patients,” he added. Here’s what to know about the issue, and how to get your blood pressure under control. What is blood pressure? And what’s a normal level? Blood pressure is the force that blood exerts against artery walls in the heart.'
recording_8_text = 'Blizzard Is Forecast to Bring ‘Life-Threatening’ Conditions to California. For the second year in a row, the Greater Lake Tahoe area is expected to begin March buried in deep, powdery, windswept snow.A rare warning for “life-threatening blizzard conditions” is in effect for the mountains of the Sierra Nevada, including Lake Tahoe, for a storm that could bring three feet of snow or more through Sunday, according to forecasters.Travel over Sierra mountain passes was already becoming treacherous early Friday morning and was expected to get more dangerous through the day and into the weekend as the blizzard worsens, warned meteorologists in the region. Blizzard warnings are reserved for the worst snow storms with whiteout conditions that could last hours or, in this case, days, with the weather likely making travel in the region treacherous. The National Weather Service in Reno, Nev., has only issued eight blizzard warnings since 2002. The last warning in the Tahoe area was almost exactly a year ago: Feb. 27, 2023. During last year’s storm, two feet of snow fell in less than 24 hours, which, combined with earlier snows'
recording_9_text = 'Top Senate Republican to Campaign with Kari Lake in Arizona John Barrasso of Wyoming joined Ms. Lake for a packed day of events in Phoenix. The pair criticized President Biden’s handling of the border crisis. Kari Lake, the former television anchor running for U.S. Senate in Arizona, spent Thursday afternoon charming supporters at a Scottsdale ballpark during a spring training game. Then she shifted tone, excoriating President Biden’s handling of the migrant crisis, which she called an “invasion,” at a news conference with Senator John Barrasso of Wyoming. “I don’t know why we have laws in Joe Biden’s world. None of the ones that protect us as American citizens are being enforced,” said Ms. Lake, rattling off a list of undocumented immigrants who had committed violent crimes in the United States. The dual appearances captured the range of Ms. Lake, whose blend of charisma and fiery rhetoric vaulted her to prominence in Arizona politics. But Ms. Lake’s divisive language in her 2022 campaign for governor pitted Republicans against one another, allowing her Democratic opponent to secure victory, and she could face similarly tough opposition this November. This time, she could have more help from the Republican establishment. The presence of Mr. Barrasso, the party’s No. 3-ranking leader, was the latest sign of the party’s embrace of a candidate whose extreme views made her a lightning rod just two years earlier. “I think our country would be much better and much safer if we had Kari Lake as a member of the United States Senate,” Mr. Barrasso said at the news conference. Earlier in the day, with both the president and former President Donald J. Trump in Texas visiting the southern border,'
recording_10_text = 'After U. of Georgia Killing, Lawmakers Seek Tougher Immigration Laws As residents lamented the killing of a 22-year-old woman, immigration policies in Georgia have come under renewed scrutiny, with state Republicans calling for stricter legislation. Republican lawmakers in Georgia are pushing to toughen state laws that govern detentions of undocumented migrants after a killing at a college campus sent shock waves through the state. Last Thursday, the body of Laken Riley, a 22-year-old woman, was found in a wooded area on the campus of the University of Georgia in Athens. According to the authorities, the man charged in her killing, Jose Antonio Ibarra, 26, is a migrant from Venezuela who crossed the Southern border in September 2022.'
data = {'file_path': ['Recording_1.wav', 'Recording_2.wav', 'Recording_3.wav', 'Recording_4.wav', 'Recording_5.wav', 'Recording_6.wav', 'Recording_7.wav', 'Recording_8.wav', 'Recording_9.wav', 'Recording_10.wav'],
        'input_text': [recording_1_text, recording_2_text, recording_3_text, recording_4_text, recording_5_text, recording_6_text, recording_7_text, recording_8_text, recording_9_text, recording_10_text],
        'output_text': [None] * 10, 
       'word_error_rate': [None] *10}

df = pd.DataFrame(data)

# Display the DataFrame
df

,file_path,input_text,output_text,word_error_rate
0,Recording_1.wav,Is There Something Radical About Painting Flow...,None,None
1,Recording_2.wav,What It’s Like to Be a Sociopath Sociopaths ar...,None,None
2,Recording_3.wav,Judge Orders Trump Removed From Illinois Prima...,None,None
3,Recording_4.wav,"In Chicago, It’s Summer in February An unseaso...",None,None
4,Recording_5.wav,"Welcome to Japan, Where the Bad News Is the Go...",None,None
5,Recording_6.wav,House passes Stopgap Bill to Head Off Shutdown...,None,None
6,Recording_7.wav,Why High Blood Pressure Matters to Your Health...,None,None
7,Recording_8.wav,Blizzard Is Forecast to Bring ‘Life-Threatenin...,None,None
8,Recording_9.wav,Top Senate Republican to Campaign with Kari La...,None,None
9,Recording_10.wav,"After U. of Georgia Killing, Lawmakers Seek To...",None,None


In [43]:
## formatting the input string so that everything is lower case and has no word in "", or any - or any .
def format_string(input_string):
    formatted_string = input_string.replace('"', '').replace('-', ' ').replace('.', '')
    return formatted_string

for i in range(10):
    df.loc[i,'input_text'] = format_string(df.loc[i,'input_text'])

df

,file_path,input_text,output_text,word_error_rate
0,Recording_1.wav,Is There Something Radical About Painting Flow...,None,None
1,Recording_2.wav,What It’s Like to Be a Sociopath Sociopaths ar...,None,None
2,Recording_3.wav,Judge Orders Trump Removed From Illinois Prima...,None,None
3,Recording_4.wav,"In Chicago, It’s Summer in February An unseaso...",None,None
4,Recording_5.wav,"Welcome to Japan, Where the Bad News Is the Go...",None,None
5,Recording_6.wav,House passes Stopgap Bill to Head Off Shutdown...,None,None
6,Recording_7.wav,Why High Blood Pressure Matters to Your Health...,None,None
7,Recording_8.wav,Blizzard Is Forecast to Bring ‘Life Threatenin...,None,None
8,Recording_9.wav,Top Senate Republican to Campaign with Kari La...,None,None
9,Recording_10.wav,"After U of Georgia Killing, Lawmakers Seek Tou...",None,None


In [44]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "cs5382-assignment-2-speech-69759869cb07.json"

In [45]:
# b. (15 points) calls the API to generate the desired output (e.g., text transcriptions of speech-to-text APIs, image classification for computer vision APIs, etc.)
from google.cloud import speech_v1p1beta1 as speech

from google.cloud import speech_v1p1beta1 as speech

def transcribe_audio(audio_path):
    client = speech.SpeechClient()

    with open(audio_path, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,
        language_code="en-US",
        enable_automatic_punctuation=False  # Disable automatic punctuation
    )

    response = client.recognize(config=config, audio=audio)

    result_string = ""
    for result in response.results:
        result_string += format(result.alternatives[0].transcript)

    return result_string


##transcribe audio:

for i in range (10):
    df.loc[i,'output_text'] = transcribe_audio(df.loc[i,'file_path'])


In [46]:
# c. (15 points) includes one code cell block analyzing the resulting output (e.g., combine your API output into one dataframe; merge it to your “ground truth” dataframe; compare the API outputs to your “ground truth” for each row; report statistics / aggregate as necessary)
# i. Comparison between ground truth and API outputs should use a reasonable “metric” that you should substantiate. As examples, Word Error Rate (WER) is a commonly used metric for speech-to-text systems; metrics like classification accuracy, precision, or recall might be relevant for facial recognition systems.
import Levenshtein

def calculate_wer(reference, hypothesis):
    # Convert the input strings to lists of words
    ref_words = reference.split()
    hyp_words = hypothesis.split()

    # Calculate Levenshtein distance
    distance = Levenshtein.distance(ref_words, hyp_words)

    # Calculate Word Error Rate
    wer = distance / len(ref_words) * 100  # WER as a percentage

    return wer



for i in range (10):
    df.loc[i,'word_error_rate'] = calculate_wer(df.loc[i,'input_text'], df.loc[i,'output_text'])


In [47]:
df

,file_path,input_text,output_text,word_error_rate
0,Recording_1.wav,Is There Something Radical About Painting Flow...,is there something radical about painting flow...,23.684211
1,Recording_2.wav,What It’s Like to Be a Sociopath Sociopaths ar...,what it's like to be a sociopath sociopaths or...,28.776978
2,Recording_3.wav,Judge Orders Trump Removed From Illinois Prima...,judge orders Trump removed from Illinois prima...,20.12987
3,Recording_4.wav,"In Chicago, It’s Summer in February An unseaso...",in Chicago with summer in February and on seas...,22.31405
4,Recording_5.wav,"Welcome to Japan, Where the Bad News Is the Go...",welcome to Japan where the bad news is a good ...,37.333333
5,Recording_6.wav,House passes Stopgap Bill to Head Off Shutdown...,how to pass a stopgap bill to head off shut do...,50.714286
6,Recording_7.wav,Why High Blood Pressure Matters to Your Health...,why high blood pressure Matters To Your Health...,26.666667
7,Recording_8.wav,Blizzard Is Forecast to Bring ‘Life Threatenin...,blizzard is forecast to bring life-threatening...,45.945946
8,Recording_9.wav,Top Senate Republican to Campaign with Kari La...,top Senate Republican to campaign with Kari La...,81.205674
9,Recording_10.wav,"After U of Georgia Killing, Lawmakers Seek Tou...",after U of Georgia killing lawmakers seek toug...,24.793388


In [49]:
#Creating dataframe for homework submission
csv_file_path = 'hw2_submission_dataframe.csv'

# Write the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

# Observations about the API performance on your data

I chose to use word error rate as my metric for judging the performance of the Google API because my research showed that word error rate is less sensitive to word order and timing variations compared to other metrics like raw accuracy. It allows for some flexibility in word ordering, making it more suitable for assessing the performance of speech recognition systems. The above code that I wrote to calculate the word error rate defines a function calculate_wer which takes in two input strings ‘reference’ and ‘hypothesis’, which then calculates the word error rte using the Levenshtein distance. As previously stated in part a), I recorded the speech from 10 different New York Times articles of various categories to have a wide range of vocabulary represented in the input speech data. From looking at the API generated text and the word error rate, one observation is that the API is not that good with hearing intonation in voice and capitalizing certain words. For example, names such as Trump was capitalized. However, several (in fact, most) words in the titles of the articles that I read aloud were not capitalized. The word error rate is quite high. It ranges from 20.13 to 81.2. However, overall 6 out of the ten recordings achieves a word error rate which is in the 20-30 range. The API performs pretty well with names, much to my surprise. My previous guess was Recording_2 and Recording_5 would be the worst performing given the presence of many names. However, Recording_2 has a relatively low word error rate at 28.78. Recording_5 has a word error rate of 37.33. Recording_5 is not one of the worst performing recordings in the dataset. The worst performing recording in the dataset is Recording_9. One reason that I can think of for this poor performance of the API is that my reading speed for this article was much faster than my reading speed for the other ones. Perhaps, a faster reading speed coupled with my accent made my speech harder to understand for the API. This disproves my hypothesis that the API would not struggle with faster speech. A few other points to note is that, the Levenshtein distance does take the difference of capitalization (the difference between ‘Hello’ and ‘hello’) into account[6]. I chose not to make all of my ground-truth data lower-case because the API does capitalize some words based on intonation and I wanted to see which words the API would capitalize. If we choose to ignore the metric and just take a glance at the generated text ignoring punctuations, capitalizations etc, the API performance can be considered "good" since most words are accurately captured. However, depending on the scenario in which the API is being used, for-example, in the scenario where a visually impaired person is trying to write an essay using this API, the API not being able to detech punctuations and capitalizations would cause issues. 

# e. Sources cited (does not need to be in ACM format), per our course policy
1. https://cloud.google.com/docs/authentication/provide-credentials-adc#local-dev
2. https://cloud.google.com/speech-to-text/docs
3. https://thepythoncode.com/article/calculate-word-error-rate-in-python
4. https://www.youtube.com/watch?v=rWcLDax-VmM 
5. https://cloud.google.com/speech-to-text/docs/speech-to-text-requests#:~:text=If%20set%2C%20Speech%2Dto%2D,limit%20length%20has%20been%20exceeded.
6. https://medium.com/@ethannam/understanding-the-levenshtein-distance-equation-for-beginners-c4285a5604f0